### 0. Prepare expression data from TCGA

In [6]:
import pandas as pd
import glob
import os
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

def addFile(file): 
	df = pd.read_csv(file, sep = "\t")
	name = os.path.basename(file)
	df['file_name'] = name
	df.columns = ['gene', 'fpkm', 'file_name']
	return(df)

def loadExpression(filepath):
	files = glob.glob(filepath)     
	dfs = (addFile(f) for f in files)
	return pd.concat(dfs, ignore_index=True)

def adjust(x): 
	return x+.0001


def topKFeatures(X, y, ifPlot = False, topK = 500):

    bestfeatures = SelectKBest(score_func=chi2, k = topK)
    fit = bestfeatures.fit(X,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)
    
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  #naming the dataframe columns
    print("print 10 best features:\n", featureScores.nlargest(10,'Score'))
    
    if ifPlot == True:
        plt.figure()
        featureScores.nlargest(10,'Score').plot(kind='barh')
        plt.show()
    
    topk_features = featureScores.nlargest(topK,'Score').Specs.values

    return topk_features

clinical = pd.read_csv("/nfs/home/jaclyns/tcga/nationwidechildrens.org_clinical_patient_prad.txt", sep="\t")
# print(labels)

mapping = pd.read_csv("/nfs/home/jaclyns/tcga/prad.txt", sep="\t")
mapping.columns = ["case_id", "aliquot_id", "file_name"]
# print(mapping)

expression = loadExpression("/nfs/home/jaclyns/tcga/prad/*FPKM-UQ.txt")
# print(expression)

labels = clinical[['bcr_patient_uuid', 'gleason_score']].merge(mapping, right_on='case_id', left_on='bcr_patient_uuid')
labels = labels[['aliquot_id', 'gleason_pattern_primary', 'gleason_pattern_secondary', 'gleason_score']]

expression = expression.merge(mapping, right_on='file_name', left_on='file_name')
expression = expression.dropna(subset=['aliquot_id'])
matrix = expression.pivot(index = 'aliquot_id', columns = 'gene', values = 'fpkm')
fmatrix = matrix.join(labels.set_index('aliquot_id'), on='aliquot_id')

preX = fmatrix.drop('gleason_score', 1)
prey = fmatrix[['gleason_score']]

topfs = topKFeatures(preX,prey, topK=1000)
features = preX[topfs]



KeyError: "['gleason_pattern_primary', 'gleason_pattern_secondary'] not in index"

# Visualization with REFINED

In this notebook we show how to convert tabular data into images, and show those images for visualization. As it is explained in the main markdown, REFINED has two steps: Initialization with manifold learning techniques (MDS), and optimization with a search technique (hill climbing). Therefore to perform visualization with REFINED, we need to do the two steps and once we get the coordinate in the square image for each feature of the tabular data we can generate images associate with each datapoint (sample).

### 1. In the below cell we perform initialization with MDS

In [3]:
import pickle
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import scipy.misc
import Toolbox
from Toolbox import two_d_eq, Assign_features_to_pixels
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import euclidean_distances
import math
import os
os.chdir('/nfs/home/jaclyns/REFINED/data')
#%% Loading the data
# empty rows I manually removed from this file
# Feat_DF = pd.read_csv("normalized_padel_feats_NCI60_672_small.csv")
#Feat_DF = pd.read_csv("C:\\Users\\obazgir\\Desktop\\CMDS_IMAGES_NEW\\normalized_padel_feats_NCI60_672.csv")
Feat_DF = features#.head(10)

X = Feat_DF.values; #X = X[:,2:]
original_input = pd.DataFrame(data = X)                              # The MDS input should be in a dataframe format with rows as samples and columns as features
feature_names_list = original_input.columns.tolist()                 # Extracting feature_names_list (gene_names or descriptor_names)
print(">>>> Data  is loaded")

#%% MDS
nn = math.ceil(np.sqrt(len(feature_names_list))) 				     # Image dimension
Nn = original_input.shape[1] 										 # Number of features
    
transposed_input = original_input.T 							     # The MDS input data must be transposed , because we want summarize each feature by two values (as compard to regular dimensionality reduction each sample will be described by two values)
Euc_Dist = euclidean_distances(transposed_input) 					 # Euclidean distance
Euc_Dist = np.maximum(Euc_Dist, Euc_Dist.transpose())   			 # Making the Euclidean distance matrix symmetric

embedding = MDS(n_components=2)										 # Reduce the dimensionality by MDS into 2 components
mds_xy = embedding.fit_transform(transposed_input)					 # Apply MDS			

print(">>>> MDS dimensionality reduction is done")

eq_xy = two_d_eq(mds_xy,Nn)
Img = Assign_features_to_pixels(eq_xy,nn,verbose=1)					# Img is the none-overlapping coordinates generated by MDS

#%% To be saved for hill climbing
Desc = Feat_DF.columns.tolist();  #Desc = Desc[2:]					# Drug descriptors name
Dist = pd.DataFrame(data = Euc_Dist, columns = Desc, index = Desc)	# Generating a distance matrix which includes the Euclidean distance between each and every descriptor
data = (Desc, Dist, Img	)  											# Preparing the hill climbing inputs

# picklename = "Init_MDS_Euc.pickle"
# picklename = "Init_PRAD_FPKM.pickle"
# with open(picklename, 'wb') as f:					# The hill climbing input is a pickle, therefore everything is saved as a pickle to be loaded by the hill climbing
#     pickle.dump(data, f)

>>>> Data  is loaded
>>>> MDS dimensionality reduction is done
>> Assign features to pixels: 347 / 1000
>> Assign features to pixels: 429 / 1000
>> Assign features to pixels: 488 / 1000
>> Assign features to pixels: 544 / 1000
>> Assign features to pixels: 593 / 1000
>> Assign features to pixels: 639 / 1000
>> Assign features to pixels: 678 / 1000
>> Assign features to pixels: 712 / 1000
>> Assign features to pixels: 736 / 1000
>> Assign features to pixels: 761 / 1000
>> Assign features to pixels: 787 / 1000
>> Assign features to pixels: 811 / 1000
>> Assign features to pixels: 834 / 1000
>> Assign features to pixels: 858 / 1000
>> Assign features to pixels: 872 / 1000
>> Assign features to pixels: 885 / 1000
>> Assign features to pixels: 896 / 1000
>> Assign features to pixels: 911 / 1000
>> Assign features to pixels: 920 / 1000
>> Assign features to pixels: 927 / 1000
>> Assign features to pixels: 934 / 1000
>> Assign features to pixels: 942 / 1000
>> Assign features to pixels: 947 /

### 2. Hill Climbing
Once the initializaiton is performed, then we have to apply the search optimizaition (hill climbing). The below bash script will run the search optimization.

In [ ]:
mpirun -np 32 python3 /nfs/home/jaclyns/REFINED/mpiHill_UF.py --init '/nfs/home/jaclyns/REFINED/data/Init_PRAD_FPKM.pickle'   --mapping 'Mapping_REFINED.pickle'  --evolution "REFINED_Evolve.csv" --num 5


### 3. Visualization
Once the search optimization is performed and the coordinates for features are obtained, we can use the coordinates to generate images for visualization purpose or training CNNs. In the below code visualization is performed.

In [26]:
#%% Visualizing REFFINED images
import math
import pickle
from Toolbox import REFINED_Im_Gen
# MDS
with open('/nfs/home/jaclyns/glow/Mapping_REFINED.pickle','rb') as file:
    gene_names_MDS,coords_MDS,map_in_int_MDS = pickle.load(file)

# # We pick the first 100 data points because of computational costs, but you can pick the entire datasets.
# # X_REFINED_MDS = REFINED_Im_Gen(X[:100,:],nn, map_in_int_MDS, gene_names_MDS,coords_MDS)
X_REFINED_MDS = REFINED_Im_Gen(X,nn, map_in_int_MDS, gene_names_MDS,coords_MDS)

# Font = 20
samples = Feat_DF.index

# aliquot = samples[1]
# print(aliquot)
# X_REFINED_MDS[0,:]

# fig=plt.figure(figsize=(12,8), dpi= 100)
# plt.imshow(X_REFINED_MDS[1,:].reshape(nn,nn), cmap = 'gray') # cmap = 'viridis')
# plt.axis('off')
# plt.savefig('/nfs/home/jaclyns/tcga/'+samples[1]+'.png')
# plt.close(fig)

for i in range(len(samples)):
    fig=plt.figure(figsize=(12,8), dpi= 100)
    plt.imshow(X_REFINED_MDS[i,:].reshape(nn,nn), cmap = 'gray') #cmap = 'viridis')
#     plt.title("Example1", fontsize = Font)
    plt.axis('off')
    fig.savefig('/nfs/home/jaclyns/tcga/refined_images/'+samples[i]+'.png')
    plt.close(fig)

# plt.subplot(142)
# plt.imshow(X_REFINED_MDS[2,:].reshape(26,26), cmap = 'viridis')
# plt.title("Example2", fontsize = Font)
# plt.axis('off')

# plt.subplot(143)
# plt.imshow(X_REFINED_MDS[3,:].reshape(26,26), cmap = 'viridis')
# plt.title("Example3", fontsize = Font)
# plt.axis('off')


# plt.subplot(144)
# plt.imshow(X_REFINED_MDS[4,:].reshape(26,26), cmap = 'viridis')
# plt.title("Example4", fontsize = Font)
# plt.axis('off')

### Converting to 16bit png
Required for Clara Train classification

In [7]:
import json
labels = prey['gleason_score'].tolist()
label1 = prey['gleason_pattern_primary'].tolist()
label2 = prey['gleason_pattern_secondary'].tolist()

dls = set(labels0)
print(dls)
dls1 = set(labels1)
print(dls1)
dls2 = set(labels2)
print(dls2)

samples = features.index

multiToBinary = {6: 0, 7: 1, 8: 2, 9: 3, 10: 4}

# clbls = [multiToBinary[x] for x in labels]
# values = {0: 0, 1: 0, 2: 0}
# for c in clbls:
#     values[c] += 1
# print(values)

training = []
for i in range(len(samples)):
    iname = "refined_images/"+samples[i]+".png"
    training.append({ "image": iname, "label": [multiToBinary[labels[i]]] })
    
init_json = { "label_format": [1], "data_format": "channels_last", "training": training }
print(json.dumps(init_json, indent=2))
# with open('/nfs/home/jaclyns/tcga/dataset.json', 'w') as outfile:
#     json.dump(init_json, outfile, indent=2)

KeyError: 'gleason_pattern_primary'